In [37]:
from typing import List

from openai import OpenAI
from dotenv import load_dotenv
import os
import glob
from docx import Document
import pandas as pd
import numpy as np
from tqdm import tqdm
from pathlib import Path
import json
import io

load_dotenv()

True

In [21]:
def load_doc(path: str, add_tables: bool = False):
    doc = Document(path)
    full_text = []
    for paragraph in doc.paragraphs:
        if len(paragraph.text) > 0:
            full_text.append(paragraph.text)
    
    text = "\n".join(full_text)
    if add_tables:
        tables_str = []
        for table_index, table in enumerate(doc.tables):
            table_str = f"Таблица {table_index + 1}"
            table_data = []
            for row in table.rows:
                row_data = [cell.text for cell in row.cells]
                table_data.append(row_data)
            markup_table = pd.DataFrame(np.vstack(table_data)).to_markdown(index=False)
            table_str += "\n" + markup_table
            tables_str.append(table_str)
        tables_full_str = "\n".join(tables_str)
        text = text + "\n" + tables_full_str
    return text

In [2]:
client = OpenAI(api_key=os.getenv("OPENAI_TOKEN"))

## Load data

In [5]:
df = pd.read_excel("./data/karti competencii_marketing.xlsx", skiprows = [0])

In [9]:
def filter_func(s: str):
    num_dots = sum([letter == "." for letter in s])
    if num_dots == 1:
        return True
    return False

In [12]:
i = 1
main_competencies = []
for value in df["Трудовые функции / компетенции"].dropna().values:
    if value.startswith(f"{i}. "):
        main_competencies.append(value.replace(f"{i}. ", ""))
        i += 1

In [36]:
print("\n".join(main_competencies))

Сбор маркетинговой информации и проведение маркетинговых исследований
Конкурентный анализ и выстраивание стратегии отличия от конкурентов 
Ассортиментный анализ и разработка товарной политики
Анализ рынка и тенденций его развития
Работа с клиентской базой
Прогнозирование спроса и продаж
Разработка конкурентных преимуществ
Разработка маркетинговой политики компании
Ценообразование и ценовая политика 
Маркетинговые и рекламные мероприятия
Разработка рекламных и информационных материалов
Генерация контента 
Вывод на рынок нового продукта
Руководство отделом/сотрудниками
Выстраивание работы с отделом продаж
Трейд-маркетинг. Разработка, реализация и совершенствование системы распределения (дистрибуции) и сбытовой политики
Управление маркетинговой деятельностью
Поисковое продвижение в интернете 
Контекстно-медийное продвижение в интернете 
Продвижение в социальных интернет-медиа (SMM)
Реализация медийной стратегии продвижения в интернете
Проведение исследований в интернете
Управление персона

In [23]:
text_task = load_doc("./data/долж_инструкция_маркетолог.docx")

In [40]:
def find_info(tech_task: str, query: str, examples: List = None):
    question = f"Какие {query} должны быть у специалиста из должностной инструкции?"
    prompt = f"Должностная инструкция:\n{tech_task}\nВопрос: {question}\nОтвет:"
    system_prompt = f"Вы эксперт в найме персонала. Необходимо извлечь необходимые {query} из должностной инструкции, "\
                    f"как отдельное поле в json с ключом '{query}'."
    if examples is not None:
        examples_str = '\n'.join(examples)
        system_prompt = system_prompt + f" Примеры {query}:\n{examples_str}"
        
    completion = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": system_prompt},
            {
                "role": "user",
                "content": prompt
            }
        ],
        response_format={ "type": "json_object"}
    )
    return completion.choices[0].message.content

In [43]:
info_competencies = find_info(text_task, "компетенции", examples=main_competencies)

In [47]:
json.loads(info_competencies)

{'компетенции': ['Сбор маркетинговой информации и проведение маркетинговых исследований',
  'Разработка комплексного плана проведения маркетингового исследования',
  'Анализ текущей рыночной конъюнктуры',
  'Составление технических заданий для выполнения маркетинговых исследований',
  'Систематизация и обобщение первичной и вторичной маркетинговой информации',
  'Прогнозирование сбыта продукции',
  'Проведение маркетинговых исследований разного типа',
  'Создание отчетов по результатам маркетингового исследования',
  'Определение конкурентоспособного ассортимента товаров и услуг',
  'Проведение тестирования инновационных товаров, услуг, брендов',
  'Разработка политики конкурентных цен на товары и услуги',
  'Расчет цен на товары и услуги',
  'Сравнение рыночных цен и цен конкурентов на товары и услуги',
  'Создание и оценка результативности каналов дистрибуции',
  'Проведение оценки сбытовой политики компании',
  'Организация внутренних и внешних коммуникационных кампаний',
  'Формиро

In [46]:
def compare_info(query: str, list_ref: List = None, list_found: List = None):
    question = f"Сопоставь два набора {query}"
    list_ref_str = '\n'.join(list_ref)
    list_found_str = '\n'.join(list_found)
    prompt = f"Существующие {query}:\n{list_ref_str}\nНайденные в должностной инструкции {query}:\n{list_found_str}\nЗапрос: {question}\nОтвет:"
    system_prompt = f"Вы эксперт в найме персонала. Необходимо сопоставить существующие {query} и найденные {query} в должностной инструкции и вывести ответ в формате таблицы markup."
    completion = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": system_prompt},
            {
                "role": "user",
                "content": prompt
            }
        ],
        # response_format={ "type": "json_object"}
    )
    return completion.choices[0].message.content

In [48]:
res = compare_info(query="компетенции", list_ref=main_competencies, list_found=json.loads(info_competencies)["компетенции"])

In [49]:
print(res)

```markup
| Существующие компетенции                                             | Найденные компетенции                                                   |
|-----------------------------------------------------------------------|-------------------------------------------------------------------------|
| Сбор маркетинговой информации и проведение маркетинговых исследований | Сбор маркетинговой информации и проведение маркетинговых исследований   |
| Конкурентный анализ и выстраивание стратегии отличия от конкурентов   | Определение конкурентоспособного ассортимента товаров и услуг            |
| Ассортиментный анализ и разработка товарной политики                  | Определение конкурентоспособного ассортимента товаров и услуг            |
| Анализ рынка и тенденций его развития                                 | Анализ текущей рыночной конъюнктуры                                      |
| Работа с клиентской базой                                             |                           